# Mesh study and validation of simulations using interface tracking
This notebook contains the plot generation for
- the mesh study (Section 3.2) 
- the comparison between numerical and experimental data (Sections 3.3.2, 3.3.3)

As experimental data, the results of the interface tracking (*image_processing_novec.ipynb*) are used. \
The interface tracking in the simulations is performed using the interfaceHeight functionObject. The resulting *.dat* files are processed to plot the interface position over time at different positions in the simulation domain.

In [ ]:
import os
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from ast import literal_eval

## Functions

In [ ]:
def get_dataframe_for_case(path_case):
    """
    Reads the openfoam output files (.dat) and converts them to a dataframe.
    
    Args:
        path_case (str): The path to the OpenFOAM case containing the .dat files.

    Returns:
        pandas.DataFrame: A dataframe containing the y and z position data in the first two rows, and the interface tracking data in the following rows.
    """

    # Define paths to the .dat files
    datFile_height = join(path_case, "postProcessing/interfaceHeight1/0/height.dat")
    datFile_position = join(path_case, "postProcessing/interfaceHeight1/0/position.dat")
    
    # Read position.dat
    df_pos_raw = pd.read_csv(datFile_position, comment='#', header=None, index_col=0,  delim_whitespace=True, nrows=1)
    df_pos_raw = df_pos_raw.replace('\)','',regex=True)
    
    # Get y and z positions
    positions_y = np.array(['y']+[df_pos_raw.iloc[0,k] for k in range(2,df_pos_raw.shape[1],4)])  
    positions_z = np.array(['z']+[df_pos_raw.iloc[0,k] for k in range(3,df_pos_raw.shape[1],4)])
    
    # Create position dataframe
    df_pos = pd.DataFrame(columns=['time'] + list(range(0,len(positions_z)-1)))
    df_pos = pd.concat([
        df_pos, 
        pd.DataFrame(positions_y.reshape(1,-1), columns=list(df_pos)),
        pd.DataFrame(positions_z.reshape(1,-1), columns=list(df_pos))
    ], ignore_index=True)
    
    
    # Create a dataframe for the interface location data (series of x values) from height.dat, drop empty lines
    df_loc = pd.read_csv(datFile_height, delim_whitespace=True, comment='#', header=None)
    drop_idx = list(range(1,df_loc.shape[1],2))
    df_loc = df_loc.drop(drop_idx, axis=1) 
    
    # Rename columns of df_loc
    columns_new = ['time']+ list(range(df_loc.shape[1]-1))
    columns_old = list(df_loc.columns)
    dict_rename = {item : columns_new[i] for i, item in enumerate(columns_old)}
    df_loc = df_loc.rename(columns=dict_rename)
    
    # Combine both dataframes
    #df_linetracking = df_pos.append(df_loc, ignore_index=True)
    df_linetracking = pd.concat([df_pos, df_loc], ignore_index=True)
   
    return df_linetracking

In [ ]:
def str_to_list_preserve_commas(s):
    """
    Converts a string representation of a list to an actual list while preserving commas.
    
    When reading a .csv file containing lists, some errors can occur in pandas.
    This function ensures correct formatting.
    Necessary to process experimental data.

     Args:
         s (str): The string representation of the list.

     Returns:
         list: The list created from the string representation.
    """
    
    s = s.replace('[ ', '[')
    s = " ".join(s.split())
    s = s.replace(' ', ',')
    data = literal_eval(s)
    
    return data

## Mesh study

**User input: paths and parameters**

In [ ]:
# Paths
path_data_experiments = r"../data_experiments"
path_data_simulations = r"../data_simulations"
path_save = r"../plots"

# chosse fluid
fluid = '02_tween' #available: 02_tween, 03_novec

# Paths for simulation cases to be considered
cases = {

    '02_tween' : [
                        join(path_data_simulations, r'02_tween/study_tween_u1e-2_d70'),   
                        join(path_data_simulations, r'02_tween/study_tween_u1e-2_d70_meshMedium'),   
                        join(path_data_simulations, r'02_tween/study_tween_u1e-2_d70_meshFine'),
                     ],    
    
    '03_novec' : [   
                        join(path_data_simulations, r'03_novec/study_novec_u1e-1_d70'),   
                        join(path_data_simulations, r'03_novec/study_novec_u1e-1_d70_meshMedium'),   
                        join(path_data_simulations, r'03_novec/study_novec_u1e-1_d70_meshFine'),
                     ],    
}

# plot cosmetics
labels=['$\Delta x = 0.033$ mm', '$\Delta x = 0.02$ mm', '$\Delta x = 0.01$ mm']
linestyles = ['--', '-.', ':']
colors=['k', 'darkblue','orange', 'g', 'g', 'lightblue', 'r']
linestyles = ['--', '-.', ':']*3
if fluid == '02_tween':
    x_cavities = 2. # position of first cavity row (mm)
    xlim = [0.15,0.38]
    ylim=[-0.2, 2.5]
else:
    x_cavities = 2. #mm
    xlim = [0,0.038]
    ylim=[-1.2, 2.5]
    
# test data frame loading
display(get_dataframe_for_case(cases[fluid][0]))

# create plot directory
os.makedirs(path_save, exist_ok=True)

**Plot**

In [ ]:
# get data for all cases
list_case_dfs = []
for c in range(len(cases[fluid])):
    df_sim = get_dataframe_for_case(cases[fluid][c])
    list_case_dfs.append([df_sim])

# loop over all three z positions, create plots
for iz in [3,2,1]:
    fig, ax = plt.subplots(figsize=(3,4.5))
    
    # loop over case data
    for c in range(len(list_case_dfs)):
        df_sim = list_case_dfs[c][0]
        
        # loop over y positions
        for iy in range(len(range(iz,df_sim.shape[1],3))):
                #print(iy)
                
            #exclude some datasets (y=4.5e-6m, y=0.0007m)
            if not iy in [0,3]:
                
                # select column to be used
                col =  range(iz,df_sim.shape[1],3)[iy]
                #print(f"y {df_sim.iloc[0,col]} z {df_sim.iloc[1,col]}")
                
                if iy == 1 :
                    # plot data, starting at row 2 (since row 0 and 1 contain the y and z positions, respectively)
                    ax.plot(df_sim.iloc[2:,0],df_sim.iloc[2:,col]*1000-x_cavities, 
                            linewidth=2, color=colors[iy], linestyle = linestyles[c], label = labels[c])

                else:
                    # plot without label
                    ax.plot(df_sim.iloc[2:,0],df_sim.iloc[2:,col]*1000-x_cavities, 
                            linewidth=1.5, color=colors[iy], linestyle = linestyles[c], label='_nolegend_')
                
    # cosmetics            
    ax.set_title(f"z = {np.around(np.float64(df_sim.iloc[1,col]),7):.2e} m")
    ax.set_xlabel("time (s)")
    ax.set_ylabel("x (mm)")
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.legend(loc='lower right')
    
    # save figure
    fig.savefig(join(path_save, f"mesh_study_interface_tracking_{fluid}_z{np.around(np.float64(df_sim.iloc[1,col]),7):.2e}.png") , bbox_inches="tight", dpi=600)

## Comparison to experimental data

### (N) Regime: Tween-air at Ca = 2.85 × 10^−4

**User input: paths, parameters**

In [ ]:
# select fluid
fluid='02_tween'

# select experimental case
case_exp = 14

# starting position of cavities for calibraton
x_cavities = 2 #mm

# offset in x and t for calibration
# (t_offset is found from visual comparison of interface position, x
# x_offset is calibrated to match the plot)
t_offset_exp = 0.052 #s 
x_offset_exp = 0.8 #mm

# paths
path_data_experiments = "../data_experiments"
path_csv_interface_tracking_exp = join(path_data_experiments, fluid, "04_interface_tracking", f"df_interface_tracking_{fluid}.csv")
path_case_sim = join(path_data_simulations, '02_tween/study_tween_u1e-2_d70')
path_save = "../plots"

# plot cosmetics
linestyles=[ "-.",(0, (1, 0.5)), "--"]
colors=['darkblue','orange', 'g', 'lightblue', 'r']
labels=['z/H = 0.01' , 'z/H = 0.5', 'z/H = 0.99' ]

# create plot directory
os.makedirs(path_save, exist_ok=True)

**Create plot**

In [ ]:
# create plot
fig, ax = plt.subplots(figsize=(4,5))

# ------------- plot simulation data
# get data
df_sim = get_dataframe_for_case(path_case_sim)

# loop over z positions for plot
for iz in [2,1,0]:

    # get data for this z and y position
    # starting at row 2 of dataframe (since row 0 and 1 contain the y and z positions, respectively)
    t = df_sim.iloc[2:,0]
    x_mm = df_sim.iloc[2:,4+iz]*1000-x_cavities
    
    # plot
    ax.plot(t,x_mm, linewidth=2 ,linestyle = linestyles[iz], label = labels[iz], color=colors[iz])

# ------------- plot experimental data

# get data
df_interface_tracking = pd.read_csv(path_csv_interface_tracking_exp, index_col='case')
t = np.array(str_to_list_preserve_commas(df_interface_tracking['list_time_range'][case_exp]))
x_top = np.array(str_to_list_preserve_commas(df_interface_tracking['list_x_min_gray_mm_top'][case_exp]))
x_bot = np.array(str_to_list_preserve_commas(df_interface_tracking['list_x_min_gray_mm_bot'][case_exp]))

# plot
ax.plot(t+t_offset_exp, (x_top+x_offset_exp)-x_cavities, '-r', linewidth=1.8, label= 'experiment, top')
ax.plot(t+t_offset_exp, (x_bot+x_offset_exp)-x_cavities, '-k', linewidth=1.8, label= 'experiment, bottom')
    
# ------------- cosmetics

ax.set_xlim(0.15,0.37)
ax.set_ylim(-0.2,2.1)
ax.set_xlabel("time (s)")
ax.set_ylabel("x (mm)")
ax.legend(loc='upper left')

# save figure
fig.savefig(join(path_save,f"validation_interface_tracking_{fluid}.png") , bbox_inches="tight", dpi=600)

### (F) Regime: Novec-air at Ca = 7.67 × 10^−3.

**User input: paths, parameters**

In [ ]:
# select fluid
fluid = '03_novec'

# select experimental case
case_exp = 2

# starting position of cavities for calibraton
x_cavities = 2 #mm

# offset in x and t for calibration 
# (t_offset is found from visual comparison of interface position, x
# x_offset is calibrated to match the plot)
t_offset_exp = 0.004285714 #s 
x_offset_exp = 0.00095 #m

# paths
path_data_experiments = "../data_experiments"
path_csv_interface_tracking_exp = join(path_data_experiments, fluid, "04_interface_tracking", f"df_interface_tracking_{fluid}.csv")
path_case_sim = join(path_data_simulations, '03_novec/study_novec_u1e-2_d70')
path_save = "../plots"

# plot cosmetics
linestyles=[ "-.",(0, (1, 0.5)), "--"]
colors=['darkblue','orange', 'g']
labels=labels=['z/H = 0.001' , 'z/H = 0.5', 'z/H = 0.999' ]

# create plot directory
os.makedirs(path_save, exist_ok=True)

**Create plot**

In [ ]:
# create plot
fig, (ax1,ax2) = plt.subplots(2, figsize=(4,6))
axes = [ax1, ax2]
        
# ------------- plot experimental data

# get data
df_interface_tracking = pd.read_csv(path_csv_interface_tracking_exp, index_col='case')
t = np.array(str_to_list_preserve_commas(df_interface_tracking['time_range'][case_exp]))
x_top = np.array(str_to_list_preserve_commas(df_interface_tracking['x_mm_front_top'][case_exp]))
x_bot = np.array(str_to_list_preserve_commas(df_interface_tracking['x_mm_front_bot'][case_exp]))
x_back_top = np.array(str_to_list_preserve_commas(df_interface_tracking['x_mm_back_top'][case_exp]))
x_back_bot = np.array(str_to_list_preserve_commas(df_interface_tracking['x_mm_back_bot'][case_exp]))

# plot
# note: experimental data are cut off at the point as the interface leaves the roi.
ax1.plot(t[:-60]+t_offset_exp, (x_bot[:-60]+x_offset_exp)*1000-x_cavities, 'k', linestyle ='-', linewidth=1.5)
ax1.plot(t[:-55]+t_offset_exp, (x_back_bot[:-55]+x_offset_exp)*1000-x_cavities,  c='k', linestyle = '-', linewidth=1.5)

ax2.plot(t[:-140]+t_offset_exp, (x_top[:-140]+x_offset_exp)*1000-x_cavities, '-', c='k', linewidth=1.5, label= 'experiment')
ax2.plot(t[:-68]+t_offset_exp, (x_back_top[:-68]+x_offset_exp)*1000-x_cavities, '-', c='k' , linewidth=1.5)
    
# ------------- plot simulation data

# get data
df_sim = get_dataframe_for_case(path_case_sim)

# loop over y positions for plot
for i in range(2):
    
    # select y position for plot
    iy = [10,4][i]
    ax = axes[i]
    
    # loop over z positions for plot
    for iz in [2,1,0]:

        # plot
        ax.plot(df_sim.iloc[2:,0],df_sim.iloc[2:,iy+iz]*1000-x_cavities, 
                linewidth= 1.5,linestyle = linestyles[iz], color=colors[iz], label=labels[iz])
        
# ------------- cosmetics
for ax in axes:
    ax.set_xlim(0.11,0.38)# 0.305)
    ax.set_ylim(-0.5,3.5)
    ax.set_ylabel("x (mm)")
ax2.set_xlabel("time (s)")
ax1.legend(loc='upper left')

# save figure
fig.savefig(join(path_save,f"validation_interface_tracking_{fluid}.png") , bbox_inches="tight", dpi=600)